# GPT Table Workflow Demo
## Purpose
This workbook provides a demo of using the GPT-4o API in order to scrape statistics from paper tables. It takes in a JAMA paper url (https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2820561), downloads images of the tables and figures from it, sends to GPT to determine if they are tables or figures, and then scrapes statistics from the figures.

## Set-up
The main packages we use are requests/bs4 for scraping, and openai, requests, and json for sending requests to the GPT API and formatting the response

In [2]:
import os

os.chdir("/Users/jackcavanagh/Documents")

import pandas as pd
import numpy as np
import openai
import json
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETIONS_MODEL = "gpt-4o"
import requests
from bs4 import BeautifulSoup

/var/folders/4j/yd1_fm0x62z55b77ssyj41y00000gn/T/ipykernel_32185/3470220223.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### Step 1: Extracting HTML from paper page
We first get the html code from the paper page so that we have links for the associated tables/figures

In [8]:
article_url = 'https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2820561'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
htm = requests.get(url = article_url,headers = headers)
soup = BeautifulSoup(htm.content, 'html.parser')

This is what the html looks like:

In [13]:
soup


<!DOCTYPE html>

<html class="page-article" id="doc" lang="en">
<head id="Head1">
<!-- charset must appear in the first 1024 bytes of the document -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script>
      var SCM = SCM || {};
      if (SCM.AMA && SCM.AMA.DataLayerAsync) {
        SCM.AMA.DataLayerAsync.push("arrSisAffinity", "", "array");
        SCM.AMA.DataLayerAsync.push("SubStatus", "no", "string");
      }

      var googletag = window.googletag || {};
      googletag.cmd = googletag.cmd || [];

          
          if (typeof Storage !== "undefined")
          {
              var targetingParamStr = localStorage.getItem("bcDFPTargetingParams");
              if (targetingParamStr)
              {
                  var targetingParameters = JSON.parse(targetingParamStr);     // set page-level targeting parameters
                  jQuery.each(targetingParameters, function (index, param) {
                      googletag.cmd.push(function () {
       

Then we get the links from the html, but we only care about links that point to images from the paper:

In [14]:
lnks = []
divs = soup.find_all("a")
for j in divs:
    if j.has_attr('href'):
        lnks.append(j['href'])
fig_tab = []
for i in lnks:
    if 'https://cdn.jamanetwork.com/ama/content_public/journal/jamanetworkopen' in i and '/downloadimage.aspx?image=' not in i:
        fig_tab.append(i)

This is what they look like:

In [19]:
fig_tab

['https://cdn.jamanetwork.com/ama/content_public/journal/jamanetworkopen/939381/zoi240554va_1719237655.21947.png?Expires=1722611483&Signature=PobBpbTwEgEm2ittvpwKUel9HXdlIJQYZm7FyzvlLQvsO7xLRQ02w44PTIaR-78MYJLa0lHFhmcfMGfVZ38LA-fSOcJ6qnImWEdtvAVSXPoJTVeqUdAIBhmxKVVuM45aFHjjM-yNONO-zsUMEZi9zN9tEPwR51-vUFM5gqcrB1qNIBs8NK8dhMLTAEmT1Qj9HRnoTiazZfRYS33v17MDvef0EUb-1--BpYR3rqTIhqWxEusq8pEk5VwEs2DQNV6KNV0VvQvgamTwYHZWQJD0gH0DhBUoHvZokn45y~gFTu9MXsY3ENGaU5OgctaabGTICzTVll9nhmgYAbUyfxpZNg__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA',
 'https://cdn.jamanetwork.com/ama/content_public/journal/jamanetworkopen/939381/zoi240554va_1719237655.21947.png?Expires=1722611483&Signature=PobBpbTwEgEm2ittvpwKUel9HXdlIJQYZm7FyzvlLQvsO7xLRQ02w44PTIaR-78MYJLa0lHFhmcfMGfVZ38LA-fSOcJ6qnImWEdtvAVSXPoJTVeqUdAIBhmxKVVuM45aFHjjM-yNONO-zsUMEZi9zN9tEPwR51-vUFM5gqcrB1qNIBs8NK8dhMLTAEmT1Qj9HRnoTiazZfRYS33v17MDvef0EUb-1--BpYR3rqTIhqWxEusq8pEk5VwEs2DQNV6KNV0VvQvgamTwYHZWQJD0gH0DhBUoHvZokn45y~gFTu9MXsY3ENGaU5OgctaabGTICzTVll9nhmgYAbUy

### Step 2: Downloading the images to a folder
We then take in the links, send a request to the site for them, and download them as pngs to a folder on our desktop:

In [20]:
os.chdir('Table_scraper')
for i in range(len(fig_tab)):
    path = f'{i}.png'
    url = fig_tab[i]
    r = requests.get(url, allow_redirects=True)
    open(path, 'wb').write(r.content)

### Step 3: Encoding the images so that they're ready for OpenAI 
It takes in base64 encoded images; this is just a function that we'll use in step 5

In [26]:
import base64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


### Step 4: Forming the request structure to send to OpenAI
The system message directs it on what to act like; the user content includes the question and the image itself. This is just a function that we'll use in the next step

In [58]:
api_key = "" ### Input your API key here

In [22]:
def form_pay(base64_image, ques):
    payload = {
  "model": "gpt-4o",
  "messages": [
      {"role": "system",
       "content": "You help answer questions about images from academic papers. Return the answer as a JSON object."
      },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": ques
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 3000,
    "response_format": { "type": "json_object" },
    }
    return payload


### Step 5: Asking OpenAI to differentiate between tables and figures
We first need to know what is a table and what is a figure without having to open all the files ourselves. This loop runs through 
all of the images we downloaded, uses the function in step 3 to encode them, and then uses the function in step 4 to form a message to send
to GPT. It then sends the message and extracts the response

In [44]:
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

imgs = os.listdir()
chk = []
for i in range(0,len(imgs)):
    base64_image = encode_image(imgs[i])
    pay1 = form_pay(base64_image, "Can you respond with whether this is a figure or a table from the paper? Respond in JSON format")
    try:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=pay1)
        check = response.json()['choices']
        js = json.loads(check[0]['message']['content'])
        js['File'] = imgs[i]
        chk.append(js)
    except:
        print('Not a table')

Not a table


This is what the response looks like -- we can see that we have four tables and a bunch of figures

In [45]:
chk

[{'type': 'table', 'File': '8.png'},
 {'type': 'table', 'File': '9.png'},
 {'type': 'figure', 'File': '15.png'},
 {'type': 'figure', 'File': '16.png'},
 {'type': 'table', 'File': '11.png'},
 {'type': 'table', 'File': '10.png'},
 {'type': 'figure', 'File': '4.png'},
 {'type': 'figure', 'File': '5.png'},
 {'type': 'figure', 'File': '7.png'},
 {'type': 'figure', 'File': '6.png'},
 {'type': 'figure', 'File': '2.png'},
 {'type': 'figure', 'File': '3.png'},
 {'type': 'figure', 'File': '1.png'},
 {'type': 'figure', 'File': '0.png'}]

### Step 6: Subsetting down to just table images

In [46]:
tabs = []
for i in chk:
    if i['type'] == 'table':
        tabs.append(i['File'])

In [47]:
tabs

['8.png', '9.png', '11.png', '10.png']

### Step 7: Sending the four table images to GPT to scrape
We use a pretty simple query ("Can you put the table in this image into formatted JSON? Only return the JSON"), but
if we wanted the tables in a very specific JSON structure we could make it more complex

In [50]:
tables = []
for i in tabs:
    base64_image = encode_image(i)
    pay1 = form_pay(base64_image, "Can you put the table in this image into formatted JSON? Only return the JSON")
    try:
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=pay1)
        check = response.json()['choices']
        js = json.loads(check[0]['message']['content'])
        tables.append(js)
    except:
        print('Didnt work')

This is what the response looks like for one of the table images

In [56]:
tables[1]

{'Table 1': {'Baseline Characteristics and Perioperative Variables': {'Group S (n = 233)': {'Age, mean (SD), y': '48.2 (10.9)',
    'Sex': {'Female': '107 (45.9)', 'Male': '126 (54.1)'},
    'BMI, mean (SD)': '23.3 (2.9)',
    'ASA health status': {'1': '73 (31.3)',
     '2': '130 (55.8)',
     '3': '30 (12.9)'},
    'ARISCAT score': {'Low risk': '62 (26.6)',
     'Moderate risk': '166 (71.2)',
     'High risk': '5 (2.1)'},
    'CCI': {'0': '146 (62.7)',
     '1-2': '56 (24.0)',
     '3-4': '22 (9.4)',
     '≥5': '5 (2.1)'},
    'Current smoker': '34 (14.6)',
    'Alcohol misuse': '10 (4.3)',
    'Hypertension': '29 (9.9)',
    'Long-term opioid use for chronic pain': '10 (4.3)',
    'Type 2 diabetes': '11 (4.7)',
    'Anesthesia time, mean (SD), h': '4.5 (1.1)',
    'Duration of surgery, mean (SD), h': '3.7 (1.0)',
    'Intraoperative opioid use, mean (SD), morphine equivalents': '27.9 (7.2)',
    'Tidal volume, mean (SD), mL/kg of predicted body weight': {'Start of surgery': '7.0 (0.

### Step 8: Turning into dataframe
We can then turn our structured JSON easily into a dataframe that has column and row names that are easy to use:

In [57]:
pd.DataFrame(tables[1]['Table 1']['Baseline Characteristics and Perioperative Variables'])

,Group S (n = 233),Group F (n = 233),Group T (n = 234)
"Age, mean (SD), y",48.2 (10.9),48.1 (10.9),47.2 (12.1)
Sex,"{'Female': '107 (45.9)', 'Male': '126 (54.1)'}","{'Female': '111 (47.6)', 'Male': '122 (52.4)'}","{'Female': '118 (50.4)', 'Male': '116 (49.6)'}"
"BMI, mean (SD)",23.3 (2.9),23.3 (2.9),23.3 (2.8)
ASA health status,"{'1': '73 (31.3)', '2': '130 (55.8)', '3': '30...","{'1': '70 (30.0)', '2': '134 (57.5)', '3': '29...","{'1': '73 (31.2)', '2': '129 (55.1)', '3': '32..."
ARISCAT score,"{'Low risk': '62 (26.6)', 'Moderate risk': '16...","{'Low risk': '64 (27.5)', 'Moderate risk': '16...","{'Low risk': '67 (28.6)', 'Moderate risk': '16..."
CCI,"{'0': '146 (62.7)', '1-2': '56 (24.0)', '3-4':...","{'0': '141 (60.5)', '1-2': '67 (28.8)', '3-4':...","{'0': '143 (61.1)', '1-2': '63 (26.9)', '3-4':..."
Current smoker,34 (14.6),34 (14.6),32 (13.7)
Alcohol misuse,10 (4.3),8 (3.4),11 (4.7)
Hypertension,29 (9.9),21 (9.0),25 (10.7)
Long-term opioid use for chronic pain,10 (4.3),12 (5.2),13 (5.6)
